In [5]:
import json
import random

with open(r"C:\Users\jizer\OneDrive\Máy tính\NLP\Assignment3\MultiWOZ2.4\data\MULTIWOZ2.4\MULTIWOZ2.4\dialogue_acts.json", "r") as file:
    dialogue_acts = json.load(file)

data = []

for dialogue_id, turns in dialogue_acts.items():
    for turn_id, acts in turns.items():
        if isinstance(acts, dict):
            for act_name, slots in acts.items():
                utterance = f"Generated utterance for {act_name}"

                dialogue_act = {
                    "act": act_name,
                    "slots": slots
                }

                data.append({
                    "utterance": utterance,
                    "dialogue_act": dialogue_act
                })
        else:
            print(f"Unexpected format in acts: {acts}")
random.shuffle(data)

total_samples = len(data)
train_size = int(0.8 * total_samples)
val_size = int(0.1 * total_samples)

train_data = data[:train_size]
val_data = data[train_size:train_size + val_size]
test_data = data[train_size + val_size:]

with open("train_data.json", "w") as train_file:
    json.dump(train_data, train_file, indent=4)
with open("val_data.json", "w") as val_file:
    json.dump(val_data, val_file, indent=4)
with open("test_data.json", "w") as test_file:
    json.dump(test_data, test_file, indent=4)


Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected format in acts: No Annotation
Unexpected forma

In [22]:
import json
from transformers import BertTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
import numpy as np
from sklearn.preprocessing import LabelEncoder


# Đọc dữ liệu từ file train_data.json
with open(r"C:\Users\jizer\OneDrive\Máy tính\NLP\Assignment3\code\December13\train_data.json", "r") as file:
    train_data = json.load(file)

# Tokenizer từ Hugging Face
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def convert_to_bio(data):
    bio_data = []
    
    for sample in data:
        utterance = sample["utterance"]
        act = sample["dialogue_act"]["act"]  # Thêm act
        slots = sample["dialogue_act"]["slots"]
        
        tokens = tokenizer.tokenize(utterance)
        tags = ["O"] * len(tokens)
        
        for slot_name, slot_value in slots:
            slot_tokens = tokenizer.tokenize(slot_value)
            for i in range(len(tokens) - len(slot_tokens) + 1):
                if tokens[i:i+len(slot_tokens)] == slot_tokens:
                    tags[i] = f"B-{slot_name}"
                    for j in range(1, len(slot_tokens)):
                        tags[i+j] = f"I-{slot_name}"
                    break

        bio_data.append({
            "tokens": tokens,
            "tags": tags,
            "act": act  
        })
    
    return bio_data
bio_train_data = convert_to_bio(train_data)

# Tạo tập hợp tất cả nhãn có thể
all_possible_tags = set()
all_possible_acts = set()



In [23]:
from transformers import BertForTokenClassification


for sample in bio_train_data:
    all_possible_tags.update(sample["tags"])
    all_possible_acts.add(sample["act"])

tag_encoder = LabelEncoder()
act_encoder = LabelEncoder()

tag_encoder.fit(list(all_possible_tags))
act_encoder.fit(list(all_possible_acts))

num_tags = len(all_possible_tags)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=num_tags)

def predict(utterance):
    # Tokenize
    tokens = tokenizer(utterance, return_tensors="pt", truncation=True, padding=True)
    
    outputs = model(**tokens)
    predictions = torch.argmax(outputs.logits, dim=-1)
    
    token_list = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
    predicted_tags = [tag_encoder.inverse_transform([p.item()])[0] for p in predictions[0][:len(token_list)]]
    
    return token_list, predicted_tags

utterance = "I want to book a hotel in Cambridge"
tokens, predicted_tags = predict(utterance)
print("Tokens:", tokens)
print("Predicted Tags:", predicted_tags)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokens: ['[CLS]', 'i', 'want', 'to', 'book', 'a', 'hotel', 'in', 'cambridge', '[SEP]']
Predicted Tags: ['O', 'O', 'O', 'B-Type', 'B-Type', 'B-Type', 'B-Type', 'B-Type', 'O', 'B-Type']


In [12]:

# Slot Filling Model
class SlotFillingModel(nn.Module):
    def __init__(self, num_tags):
        super(SlotFillingModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_tags)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        return logits

# Training function
def train_model(model, train_loader, device, optimizer, criterion, num_epochs=5):
    print('Check epoch')
    model.train()
    for epoch in range(num_epochs):
        print(num_epochs)
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            tags = batch['tags'].to(device)
            print(input_ids)
            
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Only consider actual tokens (not padding) in loss computation
            active_loss = attention_mask.view(-1) == 1
            active_logits = outputs.view(-1, outputs.shape[-1])
            active_labels = torch.where(
                active_loss,
                tags.view(-1),
                torch.tensor(criterion.ignore_index).type_as(tags)
            )
            
            loss = criterion(active_logits, active_labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            print(total_loss)
            
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")


In [33]:
import json
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Custom Dataset class
class DialogueDataset(Dataset):
    def __init__(self, data, tokenizer, tag_encoder, act_encoder, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.tag_encoder = tag_encoder
        self.act_encoder = act_encoder
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        tokens = item["tokens"]
        tags = item["tags"]
        act = item["act"]

        # Encode tokens
        encoding = self.tokenizer.encode_plus(
            tokens,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt',
            truncation=True,
            is_split_into_words=True
        )

        # Encode tags
        labels = self.tag_encoder.transform(tags)
        labels = [labels[i] if i < len(labels) else -100 for i in range(self.max_len)]
        
        # Encode act
        act_label = self.act_encoder.transform([act])[0]

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'tags': torch.tensor(labels, dtype=torch.long),
            'act': torch.tensor(act_label, dtype=torch.long)
        }

# Combined model for both slot filling and act detection
class DialogueActModel(nn.Module):
    def __init__(self, num_tags, num_acts):
        super(DialogueActModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.slot_classifier = nn.Linear(self.bert.config.hidden_size, num_tags)
        self.act_classifier = nn.Linear(self.bert.config.hidden_size, num_acts)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        pooled_output = outputs[1]
        
        sequence_output = self.dropout(sequence_output)
        pooled_output = self.dropout(pooled_output)
        
        slot_logits = self.slot_classifier(sequence_output)
        act_logits = self.act_classifier(pooled_output)
        
        return slot_logits, act_logits

# Training function with validation
def train_and_validate(model, train_loader, val_loader, device, num_epochs=5):
    slot_criterion = nn.CrossEntropyLoss(ignore_index=-100)
    act_criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    
    best_f1 = 0
    best_model = None
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            slot_labels = batch['tags'].to(device)
            act_labels = batch['act'].to(device)
            
            optimizer.zero_grad()
            slot_logits, act_logits = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Calculate slot filling loss
            active_loss = attention_mask.view(-1) == 1
            active_logits = slot_logits.view(-1, slot_logits.shape[-1])
            active_labels = torch.where(
                active_loss,
                slot_labels.view(-1),
                torch.tensor(slot_criterion.ignore_index).type_as(slot_labels)
            )
            slot_loss = slot_criterion(active_logits, active_labels)
            
            # Calculate act detection loss
            act_loss = act_criterion(act_logits, act_labels)
            
            # Combined loss
            loss = slot_loss + act_loss
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_slot_preds, val_slot_labels = [], []
        val_act_preds, val_act_labels = [], []
        
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                slot_labels = batch['tags'].to(device)
                act_labels = batch['act'].to(device)
                
                slot_logits, act_logits = model(input_ids=input_ids, attention_mask=attention_mask)
                
                # Get predictions
                slot_preds = torch.argmax(slot_logits, dim=-1)
                act_preds = torch.argmax(act_logits, dim=-1)
                
                # Collect predictions and labels for F1 calculation
                active_mask = attention_mask.view(-1) == 1
                val_slot_preds.extend(slot_preds.view(-1)[active_mask].cpu().numpy())
                val_slot_labels.extend(slot_labels.view(-1)[active_mask].cpu().numpy())
                val_act_preds.extend(act_preds.cpu().numpy())
                val_act_labels.extend(act_labels.cpu().numpy())
        
        # Calculate F1 scores
        slot_f1 = f1_score(val_slot_labels, val_slot_preds, average='weighted')
        act_f1 = f1_score(val_act_labels, val_act_preds, average='weighted')
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training Loss: {total_loss/len(train_loader):.4f}")
        print(f"Validation Slot F1: {slot_f1:.4f}")
        print(f"Validation Act F1: {act_f1:.4f}")
        
        # Save best model
        if (slot_f1 + act_f1) / 2 > best_f1:
            best_f1 = (slot_f1 + act_f1) / 2
            best_model = model.state_dict()
    
    return best_model

# Evaluation function for test set
def evaluate(model, test_loader, device, tag_encoder, act_encoder):
    model.eval()
    test_slot_preds, test_slot_labels = [], []
    test_act_preds, test_act_labels = [], []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            slot_labels = batch['tags'].to(device)
            act_labels = batch['act'].to(device)
            
            slot_logits, act_logits = model(input_ids=input_ids, attention_mask=attention_mask)
            
            slot_preds = torch.argmax(slot_logits, dim=-1)
            act_preds = torch.argmax(act_logits, dim=-1)
            
            active_mask = attention_mask.view(-1) == 1
            test_slot_preds.extend(slot_preds.view(-1)[active_mask].cpu().numpy())
            test_slot_labels.extend(slot_labels.view(-1)[active_mask].cpu().numpy())
            test_act_preds.extend(act_preds.cpu().numpy())
            test_act_labels.extend(act_labels.cpu().numpy())
    
    # Calculate final F1 scores
    slot_f1 = f1_score(test_slot_labels, test_slot_preds, average='weighted')
    act_f1 = f1_score(test_act_labels, test_act_preds, average='weighted')
    
    return slot_f1, act_f1

In [57]:
def load_data(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Đường dẫn gốc
base_path = r"C:\Users\jizer\OneDrive\Máy tính\NLP\Assignment3\code\December13"

# Load các tập dữ liệu
train_data = load_data(f"{base_path}/train_data.json")
val_data = load_data(f"{base_path}/val_data.json")
test_data = load_data(f"{base_path}/test_data.json")

In [64]:
import json
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

# Load data function
def load_data(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Custom Dataset class
class DialogueDataset(Dataset):
    def __init__(self, data, tokenizer, slot_encoder, act_encoder, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.slot_encoder = slot_encoder
        self.act_encoder = act_encoder
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        tokens = item["utterance"].split()  # assuming tokens are split by spaces
        slots = item["dialogue_act"]["slots"]
        act = item["dialogue_act"]["act"]

        # Encode tokens
        encoding = self.tokenizer.encode_plus(
            tokens,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt',
            truncation=True,
            is_split_into_words=True
        )

        # Encode slots (one slot per token)
        slot_labels = [-100] * self.max_len  # default no slot
        for i, (slot_name, slot_value) in enumerate(slots):
            # Assuming slot name is transformed to an integer label
            slot_label = self.slot_encoder.transform([slot_name])[0]
            slot_labels[i] = slot_label

        # Encode act
        act_label = self.act_encoder.transform([act])[0]

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'slots': torch.tensor(slot_labels, dtype=torch.long),
            'act': torch.tensor(act_label, dtype=torch.long)
        }

# Combined model for slot filling and act detection
class DialogueActModel(nn.Module):
    def __init__(self, num_slots, num_acts):
        super(DialogueActModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.slot_classifier = nn.Linear(self.bert.config.hidden_size, num_slots)
        self.act_classifier = nn.Linear(self.bert.config.hidden_size, num_acts)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        pooled_output = outputs[1]

        sequence_output = self.dropout(sequence_output)
        pooled_output = self.dropout(pooled_output)

        slot_logits = self.slot_classifier(sequence_output)
        act_logits = self.act_classifier(pooled_output)

        return slot_logits, act_logits

# Training function
def train_and_validate(model, train_loader, val_loader, device, num_epochs=5):
    slot_criterion = nn.CrossEntropyLoss(ignore_index=-100)
    act_criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

    for epoch in range(num_epochs):
        # Training
        model.train()
        total_loss = 0
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        with tqdm(total=len(train_loader), desc="Training") as progress_bar:
            for batch in train_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                slot_labels = batch['slots'].to(device)
                act_labels = batch['act'].to(device)

                optimizer.zero_grad()
                slot_logits, act_logits = model(input_ids=input_ids, attention_mask=attention_mask)

                # Slot loss
                active_loss = attention_mask.view(-1) == 1
                active_logits = slot_logits.view(-1, slot_logits.shape[-1])
                active_labels = torch.where(
                    active_loss,
                    slot_labels.view(-1),
                    torch.tensor(slot_criterion.ignore_index).type_as(slot_labels)
                )
                slot_loss = slot_criterion(active_logits, active_labels)

                # Act loss
                act_loss = act_criterion(act_logits, act_labels)

                # Combined loss
                loss = slot_loss + act_loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                progress_bar.update(1)
                progress_bar.set_postfix({"loss": loss.item()})

        print(f"Training Loss: {total_loss/len(train_loader):.4f}")

        # Validation
        model.eval()
        val_slot_preds, val_slot_labels = [], []
        val_act_preds, val_act_labels = [], []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                slot_labels = batch['slots'].to(device)
                act_labels = batch['act'].to(device)

                slot_logits, act_logits = model(input_ids=input_ids, attention_mask=attention_mask)

                # Get predictions
                slot_preds = torch.argmax(slot_logits, dim=-1)
                act_preds = torch.argmax(act_logits, dim=-1)

                active_mask = attention_mask.view(-1) == 1
                val_slot_preds.extend(slot_preds.view(-1)[active_mask].cpu().numpy())
                val_slot_labels.extend(slot_labels.view(-1)[active_mask].cpu().numpy())
                val_act_preds.extend(act_preds.cpu().numpy())
                val_act_labels.extend(act_labels.cpu().numpy())

        # Calculate F1 scores
        slot_f1 = f1_score(val_slot_labels, val_slot_preds, average='weighted')
        act_f1 = f1_score(val_act_labels, val_act_preds, average='weighted')

        print(f"Validation Slot F1: {slot_f1:.4f}")
        print(f"Validation Act F1: {act_f1:.4f}")

# Main script
if __name__ == "__main__":
    # Paths
    base_path = r"C:\Users\jizer\OneDrive\Máy tính\NLP\Assignment3\code\December13"
    train_data = load_data(f"{base_path}/train_data.json")
    val_data = load_data(f"{base_path}/val_data.json")
    test_data = load_data(f"{base_path}/test_data.json")

    # Tokenizer and encoders
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    slot_encoder = LabelEncoder()
    act_encoder = LabelEncoder()

    # Fit encoders
    slot_encoder.fit([slot[0] for item in train_data for slot in item['dialogue_act']['slots']])
    act_encoder.fit([item['dialogue_act']['act'] for item in train_data])

    # Datasets and DataLoaders
    train_dataset = DialogueDataset(train_data, tokenizer, slot_encoder, act_encoder, max_len=64)
    val_dataset = DialogueDataset(val_data, tokenizer, slot_encoder, act_encoder, max_len=64)
    test_dataset = DialogueDataset(test_data, tokenizer, slot_encoder, act_encoder, max_len=64)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)
    test_loader = DataLoader(test_dataset, batch_size=16)

    # Model and device
    num_slots = len(slot_encoder.classes_)
    num_acts = len(act_encoder.classes_)
    model = DialogueActModel(num_slots, num_acts)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Train and validate
    train_and_validate(model, train_loader, val_loader, device, num_epochs=5)



Epoch 1/5


Training: 100%|██████████| 5391/5391 [2:45:06<00:00,  1.84s/it, loss=1.38]   


Training Loss: 1.6016
Validation Slot F1: 0.0258
Validation Act F1: 1.0000

Epoch 2/5


Training: 100%|██████████| 5391/5391 [3:46:15<00:00,  2.52s/it, loss=1.2]       


Training Loss: 1.4141
Validation Slot F1: 0.0271
Validation Act F1: 1.0000

Epoch 3/5


Training:  89%|████████▉ | 4796/5391 [2:32:18<18:36,  1.88s/it, loss=1.62]   